## Install required libraries

In [ ]:
!pip install rembg
!pip install imutils

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 10.4 MB/s eta 0:00:00


In [ ]:
from PIL import Image, ImageFilter
import cv2
import os
import random
import imutils
from rembg import remove

## Generation of Synthetic Data ##

In [ ]:
# Folder paths for front and background images
front_folder = '/content/drive/MyDrive/FYP/comm_aircraft'
background_folder = '/content/drive/MyDrive/FYP/sky'
output_folder = '/content/drive/MyDrive/FYP/comm_in_sky'

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)
output_folder_images = os.path.join(output_folder, 'images')
output_folder_labels = os.path.join(output_folder, 'newLabels')
os.makedirs(output_folder_images, exist_ok=True)
os.makedirs(output_folder_labels, exist_ok=True)

# Process each background image
for bg_filename in os.listdir(background_folder):
    if not bg_filename.endswith('.jpg') and not bg_filename.endswith('.png') and not bg_filename.endswith('.PNG') and not bg_filename.endswith('.jpeg'):
        continue

    # Open background image
    bg_path = os.path.join(background_folder, bg_filename)
    background = Image.open(bg_path).convert("RGBA")
    background = background.resize((1920,1080))

    # Process each front image
    for front_filename in os.listdir(front_folder):
        if not front_filename.endswith('.jpg') and not front_filename.endswith('.png') and not front_filename.endswith('.PNG') and not front_filename.endswith('.jpeg') and not front_filename.endswith('.webp'):
            continue

        # Open front image
        front_path = os.path.join(front_folder, front_filename)
        frontImage = cv2.imread(front_path)

        # Resize the front image to match the background size
        front_width, front_height, _ = frontImage.shape
        if front_width < 200:
          front_width *= 2
        elif front_width < 400:
          front_width = front_width
        elif front_width < 750:
          front_width //= 2
        elif front_width < 1000:
          front_width //= 3
        else:
          front_width //= 4
        resized_frontImage = imutils.resize(frontImage, width=front_width)

        original_frontImage = resized_frontImage.copy()
        flipped_frontImage = cv2.flip(original_frontImage, 1)

        # Rotated
        angle = random.uniform(0, 10)
        rotated_frontImage = imutils.rotate_bound(original_frontImage, angle)

        # Flipped and Rotated
        flipped_rotated_frontImage = cv2.flip(original_frontImage, 1)

        # Save the resulting images
        images = {
            "original": original_frontImage,
            "flipped": flipped_frontImage,
            "rotated": rotated_frontImage,
            "flipped_rotated": flipped_rotated_frontImage
        }

        for name, img in images.items():
          # Convert image to RGBA
          Gaussian = cv2.GaussianBlur(img, (3,3), 0)
          filename = f'blurred_front_{name}.png'
          blurred_img_path = os.path.join('/content/drive/MyDrive/FYP/random', filename)
          cv2.imwrite(blurred_img_path, Gaussian)
          frontImage = Image.open(blurred_img_path)
          frontImage = remove(frontImage)
          frontImage = frontImage.convert("RGBA")

          # Calculate width and height to be at the center
          orig_width = (background.width - frontImage.width)
          orig_height = (background.height - frontImage.height) // 2
          width = random.randint(orig_width // 4, 3 * orig_width // 4)
          height = random.randint(orig_height // 4, 3 * orig_height // 4)

          # Paste the front image onto the background
          background_copy = background.copy()  # Create a copy to prevent modifying the original
          background_copy.paste(frontImage, (width, height), frontImage)
          # width and height gives the top left corner
          center_width = width + frontImage.width/2
          center_height = height + frontImage.height/2
          bbox = [center_width / 1920, center_height / 1080, frontImage.width / 1920, frontImage.height / 1080]

          # Save the resulting image
          output_filename = f'result_{bg_filename.split(".")[0]}_{front_filename.split(".")[0]}_{name}.png'
          output_folder_images = f'{output_folder}/images'
          output_folder_labels = f'{output_folder}/newLabels'
          output_path = os.path.join(output_folder_images, output_filename)
          if not os.path.exists(output_folder_images):
            os.makedirs(output_folder_images)
          background_copy.save(output_path, format="png")

          # Write the bounding box to a text file
          label_path = os.path.join(output_folder_labels, output_filename)
          label_path = os.path.splitext(label_path)[0] + '.txt'
          with open(label_path, 'w') as f:
              f.write('0 ')
              f.write(' '.join(map(str, bbox)))


print("Images processed and saved successfully.")

Images processed and saved successfully.
